In [1]:
%cd ..

D:\Programming\Research\Thesis\paramsum


In [2]:
import re
from typing import List


def clean(review: str, lowercase: bool = False, char_clean: bool = False, clean_mark_number_patterns: bool = False,
          lemmatize: bool = False, tokenize: bool = False, remove_stopwords: bool = False,
          tokenizer_type: str = None, lemmatizer_type: str = 'pymorphy2', stopwords: List[str] = None,
          tagger=None, lemmatizer=None):
    import re

    if lowercase:
        review = review.lower()

    if char_clean:
        _html_pattern = r'<[^<]+?>'
        _escape_pattern = r'[\n|\r|\b]'
        review = re.sub(_html_pattern, '  ', review)
        review = re.sub(_escape_pattern, '  ', review)

    if clean_mark_number_patterns:
        mark_number_pattern = r'\d{1,2} из \d{1,2}'
        review = re.sub(mark_number_pattern, '  ', review)

    if tokenize:
        if tokenizer_type == 'razdel':
            import razdel
            review_tokens = [token.text for token in list(razdel.tokenize(review))]

        elif tokenizer_type == 'TreebankWordTokenizer':
            from nltk.tokenize import TreebankWordTokenizer
            review_tokens = TreebankWordTokenizer().tokenize(review)

        elif tokenizer_type == 'rutokenizer':
            import rutokenizer
            tokenizer = rutokenizer.Tokenizer()
            tokenizer.load()
            review_tokens = tokenizer.tokenize(review)
        else:
            raise NotImplementedError('Unknown tokenizer')

        if lemmatize:
            if lemmatizer_type == 'pymorphy2':
                import pymorphy2
                morph = pymorphy2.MorphAnalyzer(lang='ru')
                review_tokens = [morph.parse(token)[0].normal_form for token in review_tokens]

            elif lemmatizer_type == 'rulemma':
                tags = tagger.tag(review_tokens)
                lemmas = lemmatizer.lemmatize(tags)
                review_tokens = list(map(lambda tpl: tpl[2], lemmas))
            else:
                raise NotImplementedError('Unknown lemmatizer')

        if remove_stopwords:
            review_tokens = [token for token in review_tokens if token not in stopwords]

        review = ' '.join(review_tokens)

    return review

In [3]:
import pandas as pd

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=12)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [4]:
reviews = pd.read_csv('./data/reviews.csv', index_col=0)
reviews

,review,kinopoiskId,type,date,positiveRating,negativeRating,author,title,film_id
0,«Зеленую милю» я смотрела два раза: 10 лет наз...,3221833,NEUTRAL,2023-01-28T20:58:59,4,7,Yanchessa,NaN,435
1,Период конца девяностых годов-начало двухтысяч...,3146204,POSITIVE,2022-05-06T09:26:39,13,2,Fozzy,Магия или реальность,435
2,"Очень сложно писать рецензию на этот фильм, та...",3120334,POSITIVE,2022-01-24T21:36:34,21,2,Denis2oo3,"Что происходит на миле, остается на миле",435
3,Любимая многими миллионами ценителями киноиску...,3104371,POSITIVE,2021-12-19T09:09:50,10,2,kingwayne,Живи долго и счастливо!,435
4,В нашем мире существует много разных фильмов. ...,3096653,POSITIVE,2021-12-01T05:31:18,18,2,Юлия Березина - 1765,Шедевр!,435
...,...,...,...,...,...,...,...,...,...
90642,Фильм «Ламборгини: Человек-легенда» снят в 202...,3214599,NEGATIVE,2023-01-09T01:55:11,3,1,Georgy_Olegovich,Марксисткий пасквиль против Ламборгини,969760
90643,"Эй, рагацци, вы это серьёзно, ТАК показывать и...",3211347,NEGATIVE,2022-12-29T19:07:13,48,4,Rosinanto,Американизированная версия итальянской биографии.,969760
90644,"Вообще, говоря о байопиках, стоит отметить, чт...",3209833,NEGATIVE,2022-12-24T19:23:18,72,7,Павел - 5699,Унылая повесть про яркую личность,969760
90645,"Визуальное качество, впрочем, также не безупре...",3201042,NEUTRAL,2022-11-26T19:49:37,309,28,devalmont,Ферруччо Ламборгини,969760


In [13]:
cur_reviews = reviews.query('film_id == 405609')[['review']]
cur_reviews

,review
55145,Совершенно потрясающая работа Тима Бёртона зас...
55146,"В 2010 году, из-под руки небезызвестного Тима ..."
55147,Каким вы представляете сказочный мир? Возможно...
55148,На днях захотелось посмотреть что-нибудь из фэ...
55149,Снятый по мотивам сказки фильм от Тима Бертона...
...,...
56161,В далеком 2000-м году свет увидела компьютерна...
56162,Такая мысль наверняка посещала не только любит...
56163,Наше воображение способно придумать целый волш...
56164,"Лично я получила именно то, что ожидала: краси..."


In [14]:
cur_reviews['review'] = cur_reviews['review'].parallel_apply(clean, 
    lowercase=True, char_clean=True, clean_mark_number_patterns=True,
    lemmatize=True, tokenize=True, remove_stopwords=False,
    tokenizer_type='razdel', lemmatizer_type='pymorphy2', stopwords=None,    
)

In [15]:
cur_reviews

,review
55145,совершенно потрясать работа тим бёртон заслужи...
55146,"в 2010 год , из-под рука небезызвестный тим бе..."
55147,какой вы представлять сказочный мир ? возможно...
55148,на день захотеться посмотреть что-нибудь из фэ...
55149,снять по мотив сказка фильм от тим бертон филь...
...,...
56161,в далёкий 2000-м год свет увидеть компьютерный...
56162,такой мысль наверняка посещать не только любит...
56163,наш воображение способный придумать целый волш...
56164,"лично я получить именно то , что ожидать : кра..."


In [16]:
full_film_review = cur_reviews.review.str.cat()

print(len(full_film_review))
full_film_review[:100]

2495010


'совершенно потрясать работа тим бёртон заслуживать внимание как ребёнок , так и взрослый . юмор , то'

In [17]:
full_film_review = ''

for txt in cur_reviews.review:
    full_film_review += txt

In [18]:
print(len(full_film_review))
full_film_review[:100]

2495010


'совершенно потрясать работа тим бёртон заслуживать внимание как ребёнок , так и взрослый . юмор , то'

In [19]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as LSA
from sumy.summarizers.text_rank import TextRankSummarizer as TR
from sumy.summarizers.lex_rank import LexRankSummarizer as LR
from sumy.summarizers.luhn import LuhnSummarizer as LN
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [21]:
LANGUAGE = "russian"
SENTENCES_COUNT = 1


if __name__ == "__main__":
    # url = "https://en.wikipedia.org/wiki/Automatic_summarization"
    # parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    # or for plain text files
    # parser = PlaintextParser.from_file("document.txt", Tokenizer(LANGUAGE))
    parser = PlaintextParser.from_string(full_film_review, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    
    for Summer in [TR]:
        summarizer = Summer(stemmer)
        # summarizer.stop_words = get_stop_words(LANGUAGE)

        print(f'\n\n')
        
        for sentence in summarizer(parser.document, SENTENCES_COUNT):
            print(sentence)




более тот из-за они теряться обычный актёр , в первый очередь это касаться малоопытный и абсолютно деревянный васиковский , который на фон этот компьютерный чудище и ас вроде деппа и картер ( вот кто действительно рвать и метать , несмотря на ограниченность свой персонаж ) выглядеть бледный беспомощный колода , тщетно рисовать на лоб жалкий потуга человеческий эмоция ; ни жестикуляция , ни мимика , ни экспрессия где это требоваться она не показать , вообще ничего , только неординарный внешность с мягкий и притягательный черта лицо , но на один внешность , как известно главный роль ещё никто не вытягивать … в целое от сказка кэролл и она волшебство не остаться и след , вино в это быть мочь не столько режиссёр , соблазниться на коммерчески успешный , но при это безликий кино , сколько сценарист и продюсер , но всё же и бартон ничего экстраординарный не показать , фильм получиться красочный , но при это до нельзя простой , скучный , банальный и бессмысленный в художественно-идейный пла

In [34]:
# TextRank




замолкает навсегда, исчезая в паутине кокона, но рождается вновь, и становиться бабачкой;         - в фильме есть Шляпник - это Майкл Джексон, это любой человек, не пожелавший становится взрослым, о нем можно говорить бесконечно, это и человек творчества, и человек с лучшими качествами всего того, чему учат детей - чести, храбрости, доброты, мужества, преданности, но это и взрослый, потому что у него есть воспоминания, потому что он страдает, еще он единственная трагическая фигура в этой истории, потому что он один знает, что Алиса больше не вернется, это и просто веселый безумец;         - в фильме есть магическая детская сила быть везде и нигде одновременно, быть сильным и слабым, быть добрым и злым одновременно, быть и не быть, это конечно же Чеширский кот;         - в фильме есть смерть, и это символическая смерть, единственный персонаж, который погибает в этой истории - это монстр Бармаглот.
Более того из-за них теряются обычные актеры, в первую очередь это касается малоопытной


KeyboardInterrupt



In [17]:
parser.

(<Sentence: совершенно потрясать работа тим бёртон заслуживать внимание как ребёнок , так и взрослый .>,
 <Sentence: юмор , тонкий отношение , красивый картинка , интересный сюжет и глубокий персонаж .>,
 <Sentence: вы получить здесь всё , о что мечтать , а мочь , даже немного большой .>,
 <Sentence: при просмотр картина восприятие делиться на несколько разный уровень , который органично дополнять друг друг .>,
 <Sentence: пройтись по несколько из они .>,
 <Sentence: есть картинка и звук .>,
 <Sentence: этот уровень погружать мы в сказка так глубоко , что думать о что-то земной уже не получаться .>,
 <Sentence: потрясающе наполнить кадр , внимание к деталь , соответствовать звуковой сопровождение создавать живой мир .>,
 <Sentence: есть история .>,
 <Sentence: она заставлять переживать , хотя сюжет фильм примерно понятный любой , знакомый с сказка , как и с жанр сказка в целое .>,
 <Sentence: есть персонаж .>,
 <Sentence: именно этот уровень демонстрировать мастерство тим бёртон .>,
 <

In [33]:
parser.document.sentences

(<Sentence: Совершенно потрясающая работа Тима Бёртона заслуживает внимания как детей, так и взрослых.>,
 <Sentence: Юмор, тонкие отношения, красивая картинка, интересный сюжет и глубокие персонажи.>,
 <Sentence: Вы получите здесь всё, о чём мечтали, а может, даже немного больше.>,
 <Sentence: При просмотре картины восприятия делиться на несколько разных уровней, которые органично дополняют друг друга.>,
 <Sentence: Пройдусь по нескольким из них.>,
 <Sentence: Есть картинка и звук.>,
 <Sentence: Этот уровень погружает нас в сказку так глубоко, что думать о чём-то земном уже не получается.>,
 <Sentence: Потрясающе наполненные кадры, внимание к деталям, соответствующее звуковое сопровождение создают живой мир.        Есть история.>,
 <Sentence: Она заставляет переживать, хотя сюжет фильма примерно понятен любому, знакомому со сказкой, как и с жанром сказки в целом.>,
 <Sentence: Есть персонажи.>,
 <Sentence: Именно этот уровень демонстрирует мастерство Тима Бёртона.>,
 <Sentence: В его п